In [ ]:
## RUN THIS CELL
import torch
import torchvision
import torchvision.transforms as transforms

## Create Datasets

In [ ]:
# create a transformation that changes are images to Tensors, 
# and then maps the images from the range [0, 1] to the range [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
## RUN THIS CELL
training_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform,
)

In [ ]:
## RUN THIS CELL
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform,
)

In [ ]:
training_loader = torch.utils.data.DataLoader(
    training_set,
    batch_size=4,
    shuffle=True,
    num_workers=1,
)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=4,
    shuffle=False,
    num_workers=1,
)

In [ ]:
## Classes in our dataset
classes = [
    'plane',
    'car',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

## Define our Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
## our network

class Net(nn.Module):
    
    def __init__(self):
        # fill in here
        
    def forward(self, x):
        # fill in here
        return x

In [ ]:
net = Net()

## Define the loss function and optimizer

In [ ]:
import torch.optim as optim

In [ ]:
loss_func = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Train the network

In [ ]:
NUM_EPOCHS = 1
PRINT_FREQ = 100

In [ ]:
for epoch in range(NUM_EPOCHS):
    current_loss = 0.0
    
    for i, data in enumerate(training_loader, 0):
        
        # fill in here
        
        current_loss += loss.item()
        if i % PRINT_FREQ == 0:
            print(f'[{epoch},{i}] loss: {current_loss / PRINT_FREQ:.3f}')
            current_loss = 0.0

## Test network on the training data

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        # fill in here

In [ ]:
accuracy = 100 * correct / total

In [ ]:
print(f'Accuracy on the 10000 test images: {accuracy:.2f}')

## Damn that sucks, what can we do better?

### Answer: Transfer Learning

In [ ]:
# We want to use a pretrained imagenet model and transfer it to our CIFAR task.
# First we need to recreate our datasets so that the input images to the network "look like"
# imagenet images.
transform = transforms.Compose([
    transforms.Resize([224, 224]), #imagenet images are 224x224
    transforms.ToTensor(),
    transforms.Normalize( # pretrained network was trained with these params
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225], 
    ),
])

In [ ]:
training_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform,
)

In [ ]:
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform,
)

In [ ]:
training_loader = torch.utils.data.DataLoader(
    training_set,
    batch_size=4,
    shuffle=True,
    num_workers=1,
)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=4,
    shuffle=False,
    num_workers=1,
)

## Now we pull in our pretrained network

In [ ]:
## RUN THIS CELL
vgg16 = torchvision.models.vgg16(pretrained=True)

In [ ]:
# Now we want to make it so that gradients don't get calculated for the parameters
# in the pretrained model.
for param in vgg16.parameters():
    param.requires_grad = False

In [ ]:
# Now we want to change the final fully connected layer in the model
# since imagenet has 1000 classes and we only want to predict 10

# get number of input features to last layer
num_features = vgg16.classifier[6].in_features
# overwrite existing fc with new one
vgg16.classifier[6] = nn.Linear(num_features, 10)

In [ ]:
# look at a summary of our network
import torchsummary
torchsummary.summary(vgg16, (3, 224, 224))

In [ ]:
## Now we need to copy our code for the loss function, training, etc from above
# NOTE: it would have been better practice to make that code into functions so that
# we wouldn't have to copy and paste code around.

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [ ]:
PRINT_FREQ = 20
from tqdm import tqdm
for epoch in range(NUM_EPOCHS):
    current_loss = 0.0
    pbar = tqdm(enumerate(training_loader, 0), total=len(training_loader))
    for i, data in pbar:
        
        # copy from above
        
        current_loss += loss.item()
        if i % PRINT_FREQ == 0:
            pbar.set_description(f'loss: {current_loss / PRINT_FREQ:.3f}')
            current_loss = 0.0

In [ ]:
# Test Model on Test data
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        # copy from above
        if total > 100:
            break
accuracy = 100 * correct / total
print(f'Accuracy on the {total} test images: {accuracy:.2f}')

In [ ]:
# Save model
## Save the model
torch.save(
    {
        'epoch': 1,
        'state_dict': vgg16.state_dict(),
        'optimizer' : optimizer.state_dict(),
    },
    'model.pth.tar'
)

In [ ]:
# load model
def load_model(filename):
    checkpoint = torch.load(filename, map_location='cpu')
    vgg16.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
load_model('model.pth.tar')